# Enviroment set up

In [3]:
import cv2
import numpy as np

# --- 1. CONFIGURATION ---

# Set to True to use the webcam, False to use a static image
USE_WEBCAM = False

# Your Aukey webcam is likely index 0, but it could be 1, 2, etc.
# Try changing this if 0 doesn't work.
CAMERA_INDEX = 0 

# Path to your static image (if USE_WEBCAM is False)
IMAGE_PATH = "prova.jpg" 

# Desired output resolution for your top-down map
# A 4:3 ratio is common, but you can change this
MAP_WIDTH = 800
MAP_HEIGHT = 600

# --- 2. GLOBAL VARIABLES ---

# List to store the 4 clicked points (source points)
src_points = []
# The image we will use for picking points
calibration_frame = None

# --- 3. MOUSE CALLBACK FUNCTION ---

def click_event(event, x, y, flags, params):
    """
    Handles mouse clicks. 
    Saves the (x, y) coordinates of 4 clicks.
    """
    global src_points, calibration_frame
    
    # Check if the left mouse button was clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        
        if len(src_points) < 4:
            # Add the clicked point to the list
            src_points.append((x, y))
            
            # Draw a circle on the image to show feedback
            cv2.circle(calibration_frame, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Source Image - Click 4 Corners", calibration_frame)
            print(f"Point {len(src_points)} added: ({x}, {y})")
        
        if len(src_points) == 4:
            print("All 4 points selected. Press 'c' to calculate the transform.")

# --- 4. MAIN SCRIPT ---

# --- Step 4a: Get the image for calibration ---

if USE_WEBCAM:
    print(f"Opening webcam index {CAMERA_INDEX}...")
    cap = cv2.VideoCapture(CAMERA_INDEX)
    
    # Optional: Set high resolution for your 1080p camera
    # This might slow down processing, but gives better calibration
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    
    if not cap.isOpened():
        print(f"Error: Could not open webcam index {CAMERA_INDEX}.")
        print("Try changing CAMERA_INDEX to 1 or 2.")
        exit()

    print("\nWebcam open.")
    print("Press 's' to snapshot the current frame for calibration.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting...")
            exit()
        
        # Show a small preview
        preview = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        cv2.imshow("Webcam Preview - Press 's' to snapshot", preview)
        
        if cv2.waitKey(1) & 0xFF == ord('s'):
            calibration_frame = frame.copy()
            print("Snapshot taken! Please close the preview window.")
            cv2.destroyWindow("Webcam Preview - Press 's' to snapshot")
            break
else:
    print(f"Loading image from {IMAGE_PATH}...")
    calibration_frame = cv2.imread(IMAGE_PATH)
    if calibration_frame is None:
        print(f"Error: Could not load image from {IMAGE_PATH}.")
        exit()
    print("Image loaded.")

# --- Step 4b: Select 4 corners ---

# Create a window and set the mouse callback
cv2.namedWindow("Source Image - Click 4 Corners")
cv2.setMouseCallback("Source Image - Click 4 Corners", click_event)

print("\n--- INSTRUCTIONS ---")
print("Click on the 4 corners of your robot arena in this order:")
print("  1. Top-Left")
print("  2. Top-Right")
print("  3. Bottom-Right")
print("  4. Bottom-Left")
print("After 4 clicks, press 'c' to continue.")

cv2.imshow("Source Image - Click 4 Corners", calibration_frame)

# Wait until 'c' is pressed
while True:
    if cv2.waitKey(1) & 0xFF == ord('c'):
        if len(src_points) == 4:
            break
        else:
            print("Please click exactly 4 points before pressing 'c'.")

cv2.destroyWindow("Source Image - Click 4 Corners")

# --- Step 4c: Calculate Transform Matrix ---

print("Calculating perspective transform matrix...")

# Convert points to NumPy array in float32 format
src_points_np = np.float32(src_points)

# Define the 4 destination points (the corners of our output map)
dst_points_np = np.float32([
    [0, 0],                  # Top-Left
    [MAP_WIDTH, 0],          # Top-Right
    [MAP_WIDTH, MAP_HEIGHT], # Bottom-Right
    [0, MAP_HEIGHT]          # Bottom-Left
])

# Calculate the perspective transform matrix
matrix = cv2.getPerspectiveTransform(src_points_np, dst_points_np)
print("Matrix calculated successfully!")

# --- Step 4d: Apply Transform in a Loop ---

print("\nStarting the real-time top-down feed.")
print("Press 'q' in the 'Top-Down Map' window to quit.")

try:
    while True:
        if USE_WEBCAM:
            # Read a new frame from the webcam
            ret, frame = cap.read()
            if not ret:
                print("Webcam disconnected.")
                break
            
            # Apply the perspective transform
            top_down_map = cv2.warpPerspective(frame, matrix, (MAP_WIDTH, MAP_HEIGHT))
            
        else:
            # For a static image, just apply it once
            top_down_map = cv2.warpPerspective(calibration_frame, matrix, (MAP_WIDTH, MAP_HEIGHT))
        
        # Display the result
        cv2.imshow("Top-Down Map", top_down_map)
        
        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # If using a static image, we don't need to loop.
        if not USE_WEBCAM:
            print("Static image transform complete. Press 'q' to quit.")
            cv2.waitKey(0) # Wait forever until a key is pressed
            break

finally:
    # --- 5. CLEANUP ---
    print("Cleaning up and closing windows.")
    if USE_WEBCAM:
        cap.release()
    cv2.destroyAllWindows()

Loading image from prova.jpg...
Image loaded.

--- INSTRUCTIONS ---
Click on the 4 corners of your robot arena in this order:
  1. Top-Left
  2. Top-Right
  3. Bottom-Right
  4. Bottom-Left
After 4 clicks, press 'c' to continue.
Point 1 added: (512, 224)
Point 2 added: (927, 289)
Point 3 added: (910, 584)
Point 4 added: (466, 530)
All 4 points selected. Press 'c' to calculate the transform.
Calculating perspective transform matrix...
Matrix calculated successfully!

Starting the real-time top-down feed.
Press 'q' in the 'Top-Down Map' window to quit.
Static image transform complete. Press 'q' to quit.
Cleaning up and closing windows.


# Localization
the following code should work with the real camera

In [1]:
import cv2
import numpy as np
import math

# --- CONFIGURATION (from Step 1, you'd load/reuse your matrix) ---
USE_WEBCAM = True # Keep True for live detection
CAMERA_INDEX = 0 
MAP_WIDTH = 800
MAP_HEIGHT = 600

# --- ARUCO CONFIGURATION ---
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
ARUCO_PARAMETERS = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)

THYMIO_MARKER_ID = 0 # ID of the ArUco marker on your Thymio

# --- OBSTACLE CONFIGURATION ---
# Adjust these values based on your lighting and obstacle appearance
OBSTACLE_THRESHOLD_VALUE = 200 # Pixels brighter than this become white
MIN_OBSTACLE_AREA = 50        # Minimum contour area to be considered an obstacle
MAX_OBSTACLE_AREA = 5000      # Maximum contour area

# --- GOAL CONFIGURATION (Template Matching) ---
GOAL_TEMPLATE_PATH = "goal_template.png" # Path to your small house image
GOAL_MATCH_THRESHOLD = 0.2 # How good the match needs to be (0.0 to 1.0)

# --- GLOBAL VARIABLES (for visualization/debugging) ---
thymio_position = None
thymio_orientation = None # Angle in degrees
obstacles_positions = []
goal_position = None

# --- FUNCTIONS ---

def get_thymio_pose(frame):
    """Detects the Thymio's ArUco marker and returns its (x,y) and orientation."""
    corners, ids, rejected = detector.detectMarkers(frame)

    if ids is not None and THYMIO_MARKER_ID in ids:
        # Find the index of our Thymio's marker
        thymio_marker_idx = np.where(ids == THYMIO_MARKER_ID)[0][0]
        
        # Get the corners of the detected marker
        marker_corners = corners[thymio_marker_idx][0]
        
        # Calculate the center of the marker
        center_x = int(np.mean(marker_corners[:, 0]))
        center_y = int(np.mean(marker_corners[:, 1]))
        
        # Calculate orientation: vector from bottom-left to top-left corner
        # Or, more robustly, from the center of the bottom edge to center of top edge
        # ArUco corners order: Top-left, Top-right, Bottom-right, Bottom-left
        
        # Midpoint of the bottom edge
        bottom_mid_x = int((marker_corners[2][0] + marker_corners[3][0]) / 2)
        bottom_mid_y = int((marker_corners[2][1] + marker_corners[3][1]) / 2)
        
        # Midpoint of the top edge
        top_mid_x = int((marker_corners[0][0] + marker_corners[1][0]) / 2)
        top_mid_y = int((marker_corners[0][1] + marker_corners[1][1]) / 2)
        
        # Vector from bottom-mid to top-mid
        dx = top_mid_x - bottom_mid_x
        dy = top_mid_y - bottom_mid_y
        
        # Calculate angle in degrees, 0 degrees being pointing right, increasing counter-clockwise
        # We want 0 degrees to be "up" (along y-axis), and positive clockwise rotation
        # atan2(dy, dx) gives angle from positive x-axis (right) counter-clockwise.
        # To get "up" as 0, and clockwise positive:
        # 1. Swap dx, dy -> atan2(dx, -dy) for "up" as 0, clockwise positive.
        angle_rad = math.atan2(dx, -dy) 
        angle_deg = math.degrees(angle_rad)
        
        # Normalize to 0-360 degrees if needed (atan2 gives -180 to 180)
        if angle_deg < 0:
            angle_deg += 360
            
        return (center_x, center_y), angle_deg
    
    return None, None

def detect_obstacles(frame):
    """Detects white irregular shapes as obstacles."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold
    # Pixels > OBSTACLE_THRESHOLD_VALUE become white (255), others black (0)
    _, binary_frame = cv2.threshold(gray_frame, OBSTACLE_THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_obstacles = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if MIN_OBSTACLE_AREA < area < MAX_OBSTACLE_AREA:
            # Calculate centroid of the obstacle
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                detected_obstacles.append((cx, cy))
    
    return detected_obstacles

def detect_goal(frame, template, threshold):
    """Detects the goal using template matching."""
    # Convert image to grayscale for template matching
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    # TM_CCOEFF_NORMED is robust to lighting changes
    res = cv2.matchTemplate(gray_frame, gray_template, cv2.TM_CCOEFF_NORMED)
    
    # Find the location(s) with the highest match score
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    if max_val >= threshold:
        # Get the top-left corner of the matched region
        top_left = max_loc
        h, w = gray_template.shape
        
        # Calculate the center of the matched region
        center_x = top_left[0] + w // 2
        center_y = top_left[1] + h // 2
        return (center_x, center_y)
    
    return None

# --- MAIN SCRIPT ---

# Load Goal Template (only once)
goal_template_img = cv2.imread(GOAL_TEMPLATE_PATH)
if goal_template_img is None:
    print(f"Error: Could not load goal template from {GOAL_TEMPLATE_PATH}.")
    print("Please ensure 'goal_template.png' (or .jpg) is in the same directory.")
    exit()
print(f"Goal template loaded from {GOAL_TEMPLATE_PATH}")

# You need your perspective transform matrix from Step 1.
# For this example, let's assume a dummy matrix or load a saved one.
# In a real scenario, you would run the Step 1 script once, save the matrix,
# and then load it here.
# For testing, if you don't have a matrix, replace with a valid one if possible
# or run Step 1 first and copy its output.
# If running this standalone, you'll need `matrix = cv2.getPerspectiveTransform(src_points_np, dst_points_np)`
# and `src_points_np`, `dst_points_np` defined.
# For simplicity, let's assume a pre-calculated matrix for now.
# --- DUMMY MATRIX (REPLACE WITH YOUR ACTUAL MATRIX) ---
# A 3x3 identity matrix will just return the original image
# You MUST replace this with the actual matrix you get from Step 1!
# For actual use, you'd load this from a file after saving it from step1_transform.py
matrix = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
], dtype=np.float32)
print("NOTE: Using a DUMMY IDENTITY MATRIX. Replace with your actual perspective transform matrix from Step 1!")
print("For this to work correctly, you need a proper `matrix` from `cv2.getPerspectiveTransform`.")
# --- END DUMMY MATRIX ---


# --- Initialize Video Capture ---
if USE_WEBCAM:
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920) # Set to camera's max resolution for better input
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    if not cap.isOpened():
        print(f"Error: Could not open webcam index {CAMERA_INDEX}.")
        exit()
else:
    print("USE_WEBCAM is False. This script is designed for live detection.")
    print("Please set USE_WEBCAM = True for continuous detection.")
    exit() # Exit if not using webcam, as this is a live detection script

print("\nStarting localization. Press 'q' to quit.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame from webcam. Exiting...")
            break

        # 1. Apply Perspective Transform (from Step 1)
        # This gives us our flattened, top-down map
        top_down_map = cv2.warpPerspective(frame, matrix, (MAP_WIDTH, MAP_HEIGHT))

        # 2. Detect Thymio
        thymio_position, thymio_orientation = get_thymio_pose(top_down_map)

        # 3. Detect Obstacles
        obstacles_positions = detect_obstacles(top_down_map)

        # 4. Detect Goal
        goal_position = detect_goal(top_down_map, goal_template_img, GOAL_MATCH_THRESHOLD)

        # --- VISUALIZATION (for debugging) ---
        display_frame = top_down_map.copy() # Make a copy to draw on

        # Draw Thymio
        if thymio_position is not None and thymio_orientation is not None:
            cv2.circle(display_frame, thymio_position, 10, (0, 255, 255), -1) # Yellow circle for center
            
            # Draw orientation arrow (pointing in the direction of thymio_orientation)
            arrow_length = 30
            
            # Convert orientation (0=up, clockwise positive) to OpenCV's (0=right, counter-clockwise positive)
            # This involves adjusting for the difference in '0' direction and rotation sense
            
            # OpenCV angle: 0 degrees is positive X-axis (right), increases CCW.
            # Our angle: 0 degrees is positive Y-axis (up), increases CW.
            #
            # If our angle is `alpha`, then relative to X-axis CCW:
            # `target_opencv_angle = 90 - alpha` (because 90deg CCW from X-axis is Y-axis, then subtract our CW angle)
            # Ensure it's in range -180 to 180 for math functions if needed, though sin/cos handle it.
            
            # Convert our angle (0=up, clockwise positive) to radians for math functions
            # 0 degrees (up) is -pi/2 rad from positive X-axis or 3pi/2
            # Use `math.radians()` to convert our `thymio_orientation` to radians.
            # Then adjust for the coordinate system: `y` grows down in OpenCV, `x` grows right.
            # 0 degree points up (+Y in normal math, -Y in OpenCV image coords)
            # 90 degrees points right (+X in normal math, +X in OpenCV image coords)
            
            # For 0 degrees pointing up in OpenCV (towards smaller Y), and increasing clockwise:
            # x_end = x_start + length * sin(theta_radians)
            # y_end = y_start - length * cos(theta_radians) (minus because positive Y is down)
            
            angle_rad = math.radians(thymio_orientation)
            end_x = int(thymio_position[0] + arrow_length * math.sin(angle_rad))
            end_y = int(thymio_position[1] - arrow_length * math.cos(angle_rad))
            
            cv2.arrowedLine(display_frame, thymio_position, (end_x, end_y), (0, 255, 255), 2)
            cv2.putText(display_frame, f"Thymio: ({thymio_position[0]}, {thymio_position[1]}) @ {int(thymio_orientation)} deg", 
                        (thymio_position[0] + 15, thymio_position[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        # Draw Obstacles
        for obs_pos in obstacles_positions:
            cv2.circle(display_frame, obs_pos, 15, (0, 0, 255), -1) # Red circles
            cv2.putText(display_frame, "Obstacle", (obs_pos[0] + 15, obs_pos[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

        # Draw Goal
        if goal_position is not None:
            cv2.circle(display_frame, goal_position, 15, (255, 0, 0), -1) # Blue circle
            cv2.putText(display_frame, "Goal", (goal_position[0] + 15, goal_position[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        # Display the frame
        cv2.imshow("Top-Down Map with Detections", display_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # --- CLEANUP ---
    print("Cleaning up and closing windows.")
    if USE_WEBCAM and cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()

Error: Could not load goal template from goal_template.png.
Please ensure 'goal_template.png' (or .jpg) is in the same directory.
Goal template loaded from goal_template.png
NOTE: Using a DUMMY IDENTITY MATRIX. Replace with your actual perspective transform matrix from Step 1!
For this to work correctly, you need a proper `matrix` from `cv2.getPerspectiveTransform`.

Starting localization. Press 'q' to quit.
Cleaning up and closing windows.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


How to generate the Code for the thymio

In [2]:
import cv2
import numpy as np

# --- Configuration ---

# Define the dictionary. We'll use the same one from the detection code.
# DICT_4X4_50 is a good choice: 4x4 squares, 50 unique IDs.
ARUCO_DICT_NAME = cv2.aruco.DICT_4X4_50

# The ID you want to generate.
MARKER_ID = 0

# The size of the output image in pixels (e.g., 300x300).
# Make this large enough for a high-quality print.
IMAGE_SIZE = 300

# The name of the output file.
OUTPUT_FILE = f"aruco_marker_id_{MARKER_ID}.png"

# --- Generation ---

# 1. Get the ArUco dictionary
try:
    aruco_dict = cv2.aruco.getPredefinedDictionary(ARUCO_DICT_NAME)
except AttributeError:
    print(f"Error: Unable to find ArUco dictionary. Make sure you have 'opencv-python-contrib' installed.")
    exit()

# 2. Create an empty image (numpy array)
# Note: ArUco functions in older OpenCV versions might draw directly.
# The modern way is to use `generateImageMarker`.
print(f"Generating marker with ID={MARKER_ID} from dictionary {ARUCO_DICT_NAME}...")

# 3. Generate the marker
# This function creates the marker image directly.
# The '1' argument is the border size (in bits/modules). 1 is standard.
marker_image = cv2.aruco.generateImageMarker(aruco_dict, MARKER_ID, IMAGE_SIZE, borderBits=1)

if marker_image is not None:
    # 4. Save the image to a file
    cv2.imwrite(OUTPUT_FILE, marker_image)
    print(f"Successfully saved marker to '{OUTPUT_FILE}'")
    
    # Optional: Display the marker
    cv2.imshow("ArUco Marker", marker_image)
    print("Press any key to close the preview.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Error: Could not generate ArUco marker.")

Generating marker with ID=0 from dictionary 0...
Successfully saved marker to 'aruco_marker_id_0.png'
Press any key to close the preview.


with webcam

In [2]:
import cv2
import numpy as np
import math

# --- 1. CONFIGURATION ---

# Set to True to use the webcam, False to use a static image
USE_WEBCAM = True  # <-- CHANGE THIS TO SWITCH

# Your Aukey webcam is likely index 0, but it could be 1, 2, etc.
CAMERA_INDEX = 0

# Path to your static image (if USE_WEBCAM is False)
IMAGE_PATH = "my_arena_image.jpg"  # <-- SET YOUR TEST IMAGE PATH

# --- CONFIGURATION (from Step 1) ---
# Desired output resolution for your top-down map
MAP_WIDTH = 800
MAP_HEIGHT = 600

# --- ARUCO CONFIGURATION ---
ARUCO_DICT = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
ARUCO_PARAMETERS = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)

THYMIO_MARKER_ID = 0  # ID of the ArUco marker on your Thymio

# --- OBSTACLE CONFIGURATION ---
# WE HAVE TO REASON ABOUT THE CHOICES WE WANT TO MAKE FOR THE ENVIROMENT
OBSTACLE_THRESHOLD_VALUE = 200# Pixels brighter than this become white
MIN_OBSTACLE_AREA = 50         # Minimum contour area <--
MAX_OBSTACLE_AREA = 5000       # Maximum contour area  <--

# --- GOAL CONFIGURATION (Template Matching) ---
GOAL_TEMPLATE_PATH = "goal_template.jpg"  # Path to your small house image
GOAL_MATCH_THRESHOLD = 0.2  # How good the match needs to be (0.0 to 1.0) <-- 

# --- GLOBAL VARIABLES (for visualization/debugging) ---
thymio_position = None
thymio_orientation = None  # Angle in degrees
obstacles_positions = []
goal_position = None

# --- FUNCTIONS ---

def get_thymio_pose(frame):
    """Detects the Thymio's ArUco marker and returns its (x,y) and orientation."""
    corners, ids, rejected = detector.detectMarkers(frame)

    if ids is not None and THYMIO_MARKER_ID in ids:
        thymio_marker_idx = np.where(ids == THYMIO_MARKER_ID)[0][0]
        marker_corners = corners[thymio_marker_idx][0]
        
        center_x = int(np.mean(marker_corners[:, 0]))
        center_y = int(np.mean(marker_corners[:, 1]))
        
        # ArUco corners order: Top-left, Top-right, Bottom-right, Bottom-left
        bottom_mid_x = int((marker_corners[2][0] + marker_corners[3][0]) / 2)
        bottom_mid_y = int((marker_corners[2][1] + marker_corners[3][1]) / 2)
        
        top_mid_x = int((marker_corners[0][0] + marker_corners[1][0]) / 2)
        top_mid_y = int((marker_corners[0][1] + marker_corners[1][1]) / 2)
        
        dx = top_mid_x - bottom_mid_x
        dy = top_mid_y - bottom_mid_y
        
        # Angle 0 degrees "up" (negative Y), positive clockwise
        angle_rad = math.atan2(dx, -dy) 
        angle_deg = math.degrees(angle_rad)
        
        if angle_deg < 0:
            angle_deg += 360
            
        return (center_x, center_y), angle_deg
    
    return None, None

def detect_obstacles(frame):
    """Detects white irregular shapes as obstacles."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    _, binary_frame = cv2.threshold(gray_frame, OBSTACLE_THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_obstacles = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if MIN_OBSTACLE_AREA < area < MAX_OBSTACLE_AREA:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                detected_obstacles.append((cx, cy))
    
    return detected_obstacles

def detect_goal(frame, template, threshold):
    """Detects the goal using template matching."""
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Check if template has alpha channel, remove it
    if template.shape[2] == 4:
        template = template[:, :, :3]
    
    # Check if template is grayscale, if not, convert
    if len(template.shape) > 2:
        gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    else:
        gray_template = template

    # Ensure template isn't larger than the frame
    if gray_template.shape[0] > gray_frame.shape[0] or gray_template.shape[1] > gray_frame.shape[1]:
        print("Warning: Goal template is larger than the frame. Skipping detection.")
        return None

    res = cv2.matchTemplate(gray_frame, gray_template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    if max_val >= threshold:
        top_left = max_loc
        h, w = gray_template.shape
        center_x = top_left[0] + w // 2
        center_y = top_left[1] + h // 2
        return (center_x, center_y)
    
    return None

# --- MAIN SCRIPT ---

# Load Goal Template (only once)
goal_template_img = cv2.imread(GOAL_TEMPLATE_PATH, cv2.IMREAD_UNCHANGED) # Load with alpha if present
if goal_template_img is None:
    print(f"Error: Could not load goal template from {GOAL_TEMPLATE_PATH}.")
    exit()
print(f"Goal template loaded from {GOAL_TEMPLATE_PATH}")

# --- You MUST replace this with your actual matrix from Step 1 ---
# You can save it from Step 1 using np.save("my_matrix.npy", matrix)
# And load it here using matrix = np.load("my_matrix.npy")
matrix = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
], dtype=np.float32)
print("NOTE: Using a DUMMY IDENTITY MATRIX. Replace with your actual perspective transform matrix!")
# --- END DUMMY MATRIX ---


# --- Initialize Video Capture or Load Image ---
cap = None
if USE_WEBCAM:
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    if not cap.isOpened():
        print(f"Error: Could not open webcam index {CAMERA_INDEX}.")
        exit()
    print("\nStarting localization from webcam. Press 'q' to quit.")
else:
    print(f"\nStarting localization from image: {IMAGE_PATH}")

try:
    while True:
        frame = None
        if USE_WEBCAM:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame from webcam. Exiting...")
                break
        else:
            # Load the static image
            frame = cv2.imread(IMAGE_PATH)
            if frame is None:
                print(f"Error: Could not load image from {IMAGE_PATH}.")
                break
        
        # 1. Apply Perspective Transform (from Step 1)
        top_down_map = cv2.warpPerspective(frame, matrix, (MAP_WIDTH, MAP_HEIGHT))

        # 2. Detect Thymio
        thymio_position, thymio_orientation = get_thymio_pose(top_down_map)

        # 3. Detect Obstacles
        obstacles_positions = detect_obstacles(top_down_map)

        # 4. Detect Goal
        goal_position = detect_goal(top_down_map, goal_template_img, GOAL_MATCH_THRESHOLD)

        # --- VISUALIZATION (for debugging) ---
        display_frame = top_down_map.copy()

        # Draw Thymio
        if thymio_position is not None and thymio_orientation is not None:
            cv2.circle(display_frame, thymio_position, 10, (0, 255, 255), -1) # Yellow
            angle_rad = math.radians(thymio_orientation)
            end_x = int(thymio_position[0] + 30 * math.sin(angle_rad))
            end_y = int(thymio_position[1] - 30 * math.cos(angle_rad))
            cv2.arrowedLine(display_frame, thymio_position, (end_x, end_y), (0, 255, 255), 2)
            cv2.putText(display_frame, f"Thymio: ({thymio_position[0]}, {thymio_position[1]}) @ {int(thymio_orientation)} deg", 
                        (thymio_position[0] + 15, thymio_position[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        # Draw Obstacles
        for obs_pos in obstacles_positions:
            cv2.circle(display_frame, obs_pos, 15, (0, 0, 255), -1) # Red
            cv2.putText(display_frame, "Obstacle", (obs_pos[0] + 15, obs_pos[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

        # Draw Goal
        if goal_position is not None:
            cv2.circle(display_frame, goal_position, 15, (255, 0, 0), -1) # Blue
            cv2.putText(display_frame, "Goal", (goal_position[0] + 15, goal_position[1] + 15), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        # Display the frame
        cv2.imshow("Top-Down Map with Detections", display_frame)

        # --- Loop Control ---
        if USE_WEBCAM:
            # For webcam, loop with a 1ms delay
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            # For static image, wait indefinitely for any key press
            print("Detection complete. Press any key to quit.")
            cv2.waitKey(0)
            break # Exit loop after displaying the static image

finally:
    # --- CLEANUP ---
    print("Cleaning up and closing windows.")
    if USE_WEBCAM and cap is not None and cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()

Goal template loaded from goal_template.jpg
NOTE: Using a DUMMY IDENTITY MATRIX. Replace with your actual perspective transform matrix!

Starting localization from webcam. Press 'q' to quit.
Cleaning up and closing windows.


In [ ]:
            await set_motors(node, 30, 30); await asyncio.sleep(2.0)  # droit moyen
            await set_motors(node, 90, 90); await asyncio.sleep(2.0)  # droit rapide
            await set_motors(node, -100, 100); await asyncio.sleep(2.3)  # virage gauche
            await set_motors(node, 30, 30); await asyncio.sleep(2.0)  # droit moyen
            await set_motors(node, 90, 90); await asyncio.sleep(2.0)  # droit rapide
            await set_motors(node, 100, -100); await asyncio.sleep(2.3)  # virage droit
            await set_motors(node, 0, 0); await asyncio.sleep(0.8)  # pause